# 微信聊天内容抽取
### 基于`WechatExporter`项目输出文件解析数据
- https://github.com/BlueMatthew/WechatExporter

>`WechatExporter`程序导出html模板后，
>在导出的资源目录下发，会存在聊天列表资产文件和聊天对象资产文件
>需要复制聊天对象html资产文件和改聊天对象的文件夹下方`Data`文件夹中的`js`资产文件，


### 获取上述文件后执行解析程序

In [1]:
import os
import json
from pathlib import Path  
from bs4 import BeautifulSoup
import shutil
from tqdm import tqdm
import time
import re

In [2]:


def extract_data_by_msgtype(html_code):
    # 使用Beautiful Soup解析HTML
    soup = BeautifulSoup(html_code, 'html.parser')

    # 查找样式为msg的div元素
    msg_div = soup.find('div', {'class': 'msg'})
    # 查找msgtype属性
    if msg_div and msg_div.get('msgtype'):
        
         # 提取msgtype
        msgtype = soup.find('div', {'class': 'msg'})['msgtype']

        # 根据不同的msgtype提取文本
        if msgtype == '1':  # 假设msgtype为1时是文本消息
            msg_text = soup.find('span', {'class': 'msg-text'}).text.strip()
            msg_text_element = soup.find('span', {'class': 'msg-text'})
            msg_text = f'{msg_text} {extract_text_with_emojis(msg_text_element)}'
            
        elif msgtype == '49':  # 假设msgtype为49时是包含转账信息的消息
            msg_text = soup.find('span', {'class': 'msg-text'}).text.strip()
            appinfo = soup.find('div', {'class': 'appinfo'})
            if appinfo:
                
                transfer_icon = appinfo.find('img', {'class': 'transfer-icon'})
                app_name = appinfo.find('span', {'class': 'app-name'}).text.strip()
                msg_text = f"{msg_text} - {app_name} ({transfer_icon})"
            else:
                msg_text = f"{msg_text}"
            
        elif msgtype == '47':  # 假设msgtype为47时是包含图片信息的消息
            img = soup.find('div', {'class': 'content-box'}).find('img') 
            msg_text = img
        
        elif msgtype == 'image':  # 假设msgtype为image时是包含图片信息的消息
            img = soup.find('div', {'class': 'content-box'}).find('img') 
            msg_text = img
        else:
            msg_text = "Unknown msgtype"  # 未知的msgtype值
          
        data = {
          'msgid': soup.select_one('div')['msgid'],
          'msgtype': soup.select_one('div')['msgtype'],
          'avatarimg': soup.select_one('img')['src'],
          'wxid': soup.select_one('span')['wxid'],
          'name': soup.select_one('span').text,
          'time': soup.select_one('.nt-box').text.split()[-2]+' '+soup.select_one('.nt-box').text.split()[-1],
          'msg_text': str(msg_text)
        }
    elif msg_div:
        # 提取文本
        msg_text = msg_div.text.strip()
        data = {
          'msgid': soup.select_one('div')['msgid'],
          'msgtype': 'chat-notice',
          'avatarimg': '',
          'wxid': '',
          'name': '',
          'time': '',
          'msg_text': str(msg_text)
        }
     
        

    return data


def extract_text_with_emojis(element):
    text = element.text.strip()
    # 使用正则表达式提取title属性的值作为表情标题 
    pattern = r'title="([^"]+)"'
    match = re.search(pattern, str(element))
    if match:
        return match.group(1)
    
    
# 测试函数
html_code = """
<div class="msg chat right " msgid="2166" msgtype="1">
    <div class="avatar-box">
        <img src="%E9%9C%B2%E1%A5%AB%E1%A9%A3_files/Portrait/.jpg" class="avatar">
    </div>
    <div class="nt-box"><span class="dspname right" wxid="+">张毛峰</span> 2022-02-13 15:51:56</div>
    <div class="content-box">
        <div class="triangle"></div>
        <span class="dont-break-out msg-text"><pre>你看<img src="%E9%9C%B2%E1%A5%AB%E1%A9%A3_files/Emoji/wx/Doge.png" title="[旺柴]" rawemoji="[旺柴]" class="wxemoji"></pre></span>
    </div>
</div>
"""


data = extract_data_by_msgtype(html_code)
 
print(data)

{'msgid': '2166', 'msgtype': '1', 'avatarimg': '%E9%9C%B2%E1%A5%AB%E1%A9%A3_files/Portrait/.jpg', 'wxid': '+', 'name': '张毛峰', 'time': '2022-02-13 15:51:56', 'msg_text': '你看 [旺柴]'}


现在有一个msg.js的文件内容如下，解析里面的内容

要解析`msg.js`文件中的内容，您可以使用Python来提取`msgArray`数组的内容。这个数组包含了一系列HTML消息，您可以将它们解析并提取所需的数据。以下是一个示例代码来解析并打印`msgArray`数组中的消息内容：

```python
import re

# 读取msg.js文件内容
with open('msg.js', 'r', encoding='utf-8') as file:
    js_code = file.read()

# 使用正则表达式提取msgArray数组内容
pattern = r'var msgArray = (\[.*?\]);'
matches = re.search(pattern, js_code, re.DOTALL)

if matches:
    msg_array_str = matches.group(1)
    # 移除反斜杠转义字符
    msg_array_str = msg_array_str.replace('\\', '')

    # 将提取的字符串解析为Python列表
    import ast
    msg_array = ast.literal_eval(msg_array_str)

    # 遍历msgArray数组并处理每个消息
    for msg_html in msg_array:
        # 在这里处理每个消息的HTML内容
        print(msg_html)
else:
    print("msgArray not found in the JavaScript code.")
```

这段代码将读取`msg.js`文件内容，使用正则表达式提取`msgArray`数组的内容，然后将其解析为Python列表。接下来，您可以遍历该列表，并处理每个消息的HTML内容。在这里，我只是简单地打印每个消息的HTML内容，您可以根据需要提取更多的数据或进行其他处理。

In [3]:
'''
获取输入文件夹内的所有js文件，并返回文件名全称列表
'''
def load_js(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.js':
                filename=os.path.join(root, file)
                L.append(filename)
        return L 

In [4]:
def batch(iterable, size):
    # range对象的step是size
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

# 提取加载js文件解析json文本
- data_folder:js目录
- save_folder：json.txt存储目录

In [5]:


data_folder = 'msg_extract'

save_folder = "msg_extract"
ds_path = Path(save_folder)
if ds_path.exists() is False:
    ds_path.mkdir()
 

js_files=load_js(data_folder) 
print("获取数据，成功{}".format(len(js_files)))

获取数据，成功1


In [6]:

batch_len = 10

#对每一个文件进行操作
batch_list = batch(js_files,batch_len)
b_unit = tqdm(enumerate(js_files),total=len(js_files))
for batch_file in batch_list:  
        
    # 更新进度
    b_unit.update(batch_len)
    table_data=[]
    for filename in batch_file:
        # 使用os.path.splitext()分割文件名和扩展名
        file_name, file_extension = os.path.splitext(os.path.basename(filename))
        save_name = os.path.join(save_folder, f"{file_name}.txt")

        if os.path.exists(save_name):
            continue
        # 读取msg.js文件内容
        with open(filename, 'r', encoding='utf-8') as file:
            js_code = file.read()

        # 使用正则表达式提取msgArray数组内容
        pattern = r'var msgArray = (\[.*?\]);'
        matches = re.search(pattern, js_code, re.DOTALL)
        msg_data = []
        if matches:
            msg_array_str = matches.group(1)
            # 移除反斜杠转义字符
            msg_array_str = msg_array_str.replace('\\t', '')

            # 将提取的字符串解析为Python列表
            import ast
            msg_array = ast.literal_eval(msg_array_str)

            # 遍历msgArray数组并处理每个消息
            for msg_html in msg_array:
                # 在这里处理每个消息的HTML内容
                msg_html_encoded = msg_html.encode('utf-8', 'ignore')  # 忽略无效字符
                msg_html_decoded = msg_html_encoded.decode('utf-8')
                data = extract_data_by_msgtype(msg_html_decoded)
                msg_data.append(data)

        else:
            print("msgArray not found in the JavaScript code.")


        with open(save_name, 'w', encoding='utf-8') as f:

            for chat in msg_data: 
                json_str = json.dumps(chat, ensure_ascii=False)
                f.write(json_str+"\n")







  0%|                                                                                                                            | 0/1 [00:00<?, ?it/s]

# 读取msg.html文件内容


有个html文件，内容如下，提取id标识为msgs-div的内容，node转换成html字符串数组


soup = BeautifulSoup(html_code, 'html.parser')


<body uid="" sid="">
		<div class="main">
			<div class="header">
				<span class="sname">露ᥫᩣ</span>
				<div class="msgfilter">
					<!-- filter.html -->

				</div>
			</div>
			<div class="msgs" id="msgs-div">

要提取HTML文件中id标识为"msgs-div"的内容并将其转换为HTML字符串数组，您可以使用Beautiful Soup来完成。首先，确保您已经安装了Beautiful Soup库。然后，可以按照以下方式进行操作：

```python
from bs4 import BeautifulSoup

# 您的HTML代码
html_code = """
<body uid="" sid="">
	<div class="main">
		<div class="header">
			<span class="sname">露ᥫᩣ</span>
			<div class="msgfilter">
				<!-- filter.html -->
			</div>
		</div>
		<div class="msgs" id="msgs-div">
			<!-- 这是msgs-div的内容 -->
			<p>第一段</p>
			<p>第二段</p>
			<p>第三段</p>
		</div>
	</div>
</body>
"""

# 使用Beautiful Soup解析HTML
soup = BeautifulSoup(html_code, 'html.parser')

# 找到id为"msgs-div"的div元素
msgs_div = soup.find(id='msgs-div')

# 将内容转换为HTML字符串数组
html_strings = []
for element in msgs_div.contents:
    if element.name:  # 过滤掉非标签元素
        html_strings.append(str(element))

# 打印HTML字符串数组
for html_string in html_strings:
    print(html_string)
```

上述代码首先找到id为"msgs-div"的div元素，然后将其内容转换为HTML字符串数组。最后，打印HTML字符串数组中的每个元素。您可以根据需要进一步处理这些HTML字符串。

In [7]:


with open('/media/gpt4-pdf-chatbot-langchain/InterpretationofDreams/露露/露ᥫᩣ.html', 'r', encoding='utf-8') as file:
    html_code = file.read()

# 使用Beautiful Soup解析HTML
soup = BeautifulSoup(html_code, 'html.parser')

# 找到id为"msgs-div"的div元素
msgs_div = soup.find(id='msgs-div')

# 将内容转换为HTML字符串数组
html_strings = []
for element in msgs_div.contents:
    if element.name:  # 过滤掉非标签元素
        html_strings.append(str(element))

msg_data = []
# 打印HTML字符串数组
for html_string in html_strings:
    # 在这里处理每个消息的HTML内容
    msg_html_encoded = html_string.encode('utf-8', 'ignore')  # 忽略无效字符
    msg_html_decoded = msg_html_encoded.decode('utf-8')
    data = extract_data_by_msgtype(msg_html_decoded)
    msg_data.append(data)


save_name = os.path.join(save_folder, f"msg-0.txt")
with open(save_name, 'w', encoding='utf-8') as f:

    for chat in msg_data: 
        json_str = json.dumps(chat, ensure_ascii=False)
        f.write(json_str+"\n")


    